In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, udf, to_date, abs as abs_
from pyspark.sql.types import IntegerType

spark = SparkSession.builder \
    .appName("Music Data Cleaning") \
    .getOrCreate()

# Carregue seu arquivo parquet
df = spark.read.parquet("/spark-data/meuarquivo.parquet")

25/07/09 01:17:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
from pyspark.sql.functions import col, lower, split, substring

# Vamos imprimir a contagem inicial de linhas para comparação
initial_count = df.count()
print(f"Número inicial de linhas: {initial_count}")

# --- Passo 1: Limpar dados duplicados ---
# Remove linhas onde a combinação de artista, álbum e música é idêntica.
df_no_duplicates = df.dropDuplicates(['Artist(s)', 'Album', 'song'])
print(f"Linhas após remover duplicatas: {df_no_duplicates.count()}")

Número inicial de linhas: 551443


[Stage 4:============================================>              (3 + 1) / 4]

Linhas após remover duplicatas: 498052


In [13]:
# --- Passo 2: Tratar dados nulos ---
# Remove linhas onde colunas essenciais como 'song', 'Album' ou 'Time signature' são nulas.
df_no_nulls = df_no_duplicates.na.drop(subset=['song', 'Album', 'Time signature'])
print(f"Linhas após remover nulos: {df_no_nulls.count()}")

[Stage 12:>                                                         (0 + 1) / 1]

Linhas após remover nulos: 498021


In [14]:
# --- Passo 3: Limpar e transformar colunas ---
df_transformed = df_no_nulls.withColumn(
    # Deixar 'Release Date' apenas com o ano
    "release_year",
    substring(col("Release Date"), -4, 4).cast("int")
).withColumn(
    # Deixar 'Genre' apenas com o primeiro valor
    "main_genre",
    split(col("Genre"), ",")[0]
).withColumn(
    # Descaptalizar os valores da coluna 'emotion'
    "emotion_lower",
    lower(col("emotion"))
).withColumn(
    # Loudness from string to float
    "Loudness (db)",
    abs_(regexp_replace(col("Loudness (db)"), "db", "").cast("double"))
) 

In [15]:
# --- Deixa as colunas em formato numerico
numeric_cols = [
    "Tempo", "Energy", "Danceability", "Positiveness",
    "Speechiness", "Liveness", "Acousticness", "Instrumentalness", "Popularity"
]

for c in numeric_cols:
    df_transformed = df_transformed.withColumn(c, col(c).cast("integer"))

In [16]:
# --- Deixar apenas uma forma de escrita de hip-hop na coluna genero ---
df_transformed = df_transformed.withColumn("main_genre", 
    when(df_transformed["main_genre"] == "hip hop", "hip-hop")
    .otherwise(df_transformed["main_genre"])
)

In [17]:
# --- Passo 4: Apagar emoções com poucos valores ---
# Lista de emoções a serem removidas (já em minúsculas)
emotions_to_remove = ['true', 'pink', 'thirst', 'angry', 'confusion', 'interest']

df_filtered = df_transformed.filter(
    ~col("emotion_lower").isin(emotions_to_remove)
)
print(f"Linhas após filtrar emoções: {df_filtered.count()}")

[Stage 18:>                                                         (0 + 1) / 1]

Linhas após filtrar emoções: 497998


In [18]:
# --- Passo 5: Apagar colunas que não vamos usar ---
columns_to_drop = [
    # Colunas originais que foram transformadas
    'Release Date', 'Genre', 'emotion',
    # Colunas que o usuário pediu para apagar
    'song',
    'Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
    'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2',
    'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'
]

df_final = df_filtered.drop(*columns_to_drop)

In [19]:
# --- Passo 5: Apagar colunas que não vamos usar ---
columns_to_drop = [
    # Colunas originais que foram transformadas
    'Release Date', 'Genre', 'emotion',
    # Colunas que o usuário pediu para apagar
    'song',
    'Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
    'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2',
    'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'
]

df_final = df_filtered.drop(*columns_to_drop)


# --- Resultado Final ---
print("\nEsquema do DataFrame final:")
df_final.printSchema()

print("\nAmostra dos dados limpos e pré-processados:")
df_final.show(10)


Esquema do DataFrame final:
root
 |-- Artist(s): string (nullable = true)
 |-- text: string (nullable = true)
 |-- Length: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Key: string (nullable = true)
 |-- Tempo: integer (nullable = true)
 |-- Loudness (db): double (nullable = true)
 |-- Time signature: string (nullable = true)
 |-- Explicit: string (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- Energy: integer (nullable = true)
 |-- Danceability: integer (nullable = true)
 |-- Positiveness: integer (nullable = true)
 |-- Speechiness: integer (nullable = true)
 |-- Liveness: integer (nullable = true)
 |-- Acousticness: integer (nullable = true)
 |-- Instrumentalness: integer (nullable = true)
 |-- Good for Party: string (nullable = true)
 |-- Good for Work/Study: string (nullable = true)
 |-- Good for Relaxation/Meditation: string (nullable = true)
 |-- Good for Exercise: string (nullable = true)
 |-- Good for Running: string (nullable = true)
 |-

[Stage 22:===========================================>              (3 + 1) / 4]

+--------------------+--------------------+------+--------------------+------+-----+-------------+--------------+--------+----------+------+------------+------------+-----------+--------+------------+----------------+--------------+-------------------+------------------------------+-----------------+----------------+------------------------+----------------+--------------------------+------------------------+------------+----------+-------------+
|           Artist(s)|                text|Length|               Album|   Key|Tempo|Loudness (db)|Time signature|Explicit|Popularity|Energy|Danceability|Positiveness|Speechiness|Liveness|Acousticness|Instrumentalness|Good for Party|Good for Work/Study|Good for Relaxation/Meditation|Good for Exercise|Good for Running|Good for Yoga/Stretching|Good for Driving|Good for Social Gatherings|Good for Morning Routine|release_year|main_genre|emotion_lower|
+--------------------+--------------------+------+--------------------+------+-----+-------------+

In [20]:
# Supondo que seu DataFrame final e limpo se chama 'df_final'

# 1. Defina o caminho de saída para o novo dataset
caminho_saida = "/spark-data/musicas_limpas_cluster.parquet"

# 2. Salve o DataFrame no formato Parquet
df_final.write.mode("overwrite").parquet(caminho_saida)

print(f"DataFrame limpo foi salvo com sucesso em: {caminho_saida}")

[Stage 27:===================================================>     (9 + 1) / 10]

DataFrame limpo foi salvo com sucesso em: /spark-data/musicas_limpas_cluste.parquet


In [10]:
spark.stop()